In [7]:
#importamos la librería simpy y otras que nos pueden ser útiles
import simpy
import numpy as np
import statistics as stats

ModuleNotFoundError: No module named 'simpy'

En primer lugar vamos a hacer el set up básico de un entorno para la simulación que incluye:

In [ ]:
# 1) SEMILLA Y ENTORNO =========================
# 👉 Environment: el "reloj" y la agenda de eventos.
seed = 42
rng = np.random.default_rng(seed)
env = simpy.Environment()

# 2) RECURSOS Y ESTRUCTURAS ====================
# 👉 Resource: capacidad limitada (agentes que atienden pedidos).
#    capacity=2 significa que pueden atenderse 2 pedidos a la vez.
agents = simpy.Resource(env, capacity=2)

# 👉 Container: inventario continuo (unidades disponibles).
#    get(q) saca y espera si no hay; put(q) repone.
inventory = simpy.Container(env, init=3, capacity=999)

# 👉 Store: cola de objetos (aquí "pedidos").
#    put(obj) encola, get() desenfila.
inbox = simpy.Store(env, capacity=999)

Ahora definiremos los parámetro del modelo, lo cual es lo que da la utilidad a la hora de estudiar como se comportan fenómenos que queremos estudiar. Los parámetros se pueden modificar experimentalmente para estudiar el comportamiento del modelo en diferentes ocasiones.

Siéntete libre de modificar los parámetros tanto como quieras para estudiar el comportamiento del modelo.

In [ ]:
# 3) PARÁMETROS DEL MODELO =====================
MEAN_INTERARRIVAL = 1.5     # tiempo medio entre llegadas
MEAN_SERVICE = 1.0          # tiempo de servicio medio
WARMUP = 0.0                # para demo, sin warmup
HORIZON = 25                # tiempo total de simulación
RESTOCK_EVERY = 5           # cada cuánto reabastecemos
RESTOCK_QTY = 3
SETUP_TIME = 0.2            # tiempo de preparación (lo usamos en AllOf)
MEAN_PACIENCE = 1.0         # paciencia promedio de un pedido (cuanto tiempo está dispuesto a esperar
                            # antes de irse sin ser atendido)

Consiguientemente añadimos las variables (métricas) en donde vamos a guardar los resultados de la simulación. En este caso Tenemos pocas métricas, pero estas se pueden ajustar a la necesidad de cada proyecto para obtener los datos necesarios.

In [ ]:
# Métricas simples
served, abandoned = 0, 0
wait_times, sojourn_times = [], []

Ahora viene una parte fundamental de cualquier simulación: definir cuales van a ser los procesos que "producen" a los eventos

In [ ]:
def arrivals(env, inbox, rng):
    """Genera pedidos y los deja en la cola (Store)."""
    i = 0
    while True:
        # 👉 env.timeout(dt): espera 'dt' unidades de tiempo simulado.
        inter = rng.exponential(MEAN_INTERARRIVAL)
        yield env.timeout(inter)
        i += 1
        patience = rng.exponential(MEAN_PACIENCE)
        item = {"id": i, "arrival": env.now, "patience": patience}
        print(f"[{env.now:5.2f}] Llega pedido {i} (paciencia ~ {patience:0.2f})")
        # 👉 store.put(obj): encola un objeto (no bloquea si hay espacio).
        yield inbox.put(item)

def dispatcher(env, inbox, agents, inventory, rng):
    """Toma pedidos de la cola y lanza su atención (customer process)."""
    while True:
        # 👉 store.get(): espera hasta que haya un objeto y lo toma.
        order = yield inbox.get()
        # 👉 env.process(gen): registra un nuevo proceso.
        env.process(customer(env, order, agents, inventory, rng))

def customer(env, order, agents, inventory, rng):
    """Proceso por pedido: espera agente, consume inventario y es atendido."""
    global served, abandoned, wait_times, sojourn_times

    arrival = order["arrival"]
    patience = order["patience"]

# 👉 Resource.request(): solicita turno en el recurso (agentes).
    req = agents.request()

    # 👉 AnyOf: espera EITHER "me dan el recurso" OR "se me acaba la paciencia".
    #    Usamos el operador '|' como atajo de SimPy para AnyOf.
    results = yield req | env.timeout(patience)

    if req not in results:
        # No obtuvo el recurso antes de perder la paciencia → abandona.
        abandoned += 1
        print(f"[{env.now:5.2f}] Pedido {order['id']} ABANDONA (impaciente)")
        return

    # A partir de aquí, SÍ consiguió el recurso (está "dentro" del servidor).
    wait = env.now - arrival
    wait_times.append(wait)

    # 👉 AllOf: exige dos condiciones en paralelo:
    #    a) Preparación mínima (setup),
    #    b) Disponer de 1 unidad de inventario.
    #    Si falta inventario, get(1) bloquea hasta reabasto.
    need = [env.timeout(SETUP_TIME), inventory.get(1)]
    yield simpy.events.AllOf(env, need)

    # 👉 Servicio: tiempo exponencial
    service = rng.exponential(MEAN_SERVICE)
    yield env.timeout(service)

    # 👉 Liberar el recurso (agente) al terminar
    agents.release(req)

    # Métricas finales del pedido
    served += 1
    sojourn = env.now - arrival
    sojourn_times.append(sojourn)
    print(f"[{env.now:5.2f}] Pedido {order['id']} servido "
          f"(espera={wait:0.2f}, total={sojourn:0.2f})")

def restocker(env, inventory):
    """Reabastece el inventario periódicamente."""
    while True:
        yield env.timeout(RESTOCK_EVERY)
        # 👉 container.put(q): repone q unidades
        yield inventory.put(RESTOCK_QTY)
        print(f"[{env.now:5.2f}] Reabasto +{RESTOCK_QTY} (stock={inventory.level:.0f})")

def maintenance(env, target_process):
    """Muestra cómo interrumpir un proceso en marcha."""
    yield env.timeout(12)
    print(f"[{env.now:5.2f}] *** Mantenimiento: interrumpimos proceso de vigilancia ***")
    target_process.interrupt("pausa programada")

def watchdog(env):
    """Proceso que 'vive' y puede ser interrumpido."""
    try:
        while True:
            yield env.timeout(3)
            print(f"[{env.now:5.2f}] Watchdog ok")
    except simpy.Interrupt as e:
        print(f"[{env.now:5.2f}] Watchdog interrumpido -> motivo: {e.cause}")
        yield env.timeout(2)
        print(f"[{env.now:5.2f}] Watchdog reanudado")

Una vez que ya tenemos la estructura del funcionamiento de la simulación, debemos registrar los procesos iniciales.

In [ ]:
env.process(arrivals(env, inbox, rng))
env.process(dispatcher(env, inbox, agents, inventory, rng))

wd = env.process(watchdog(env))     # proceso "interrumpible"
env.process(maintenance(env, wd))   # plan de interrupción

env.process(restocker(env, inventory))  # reabastecimientos periódicos

Y por último, corremos la simulación.

In [ ]:
print(f"== Inicio simulación (horizonte={HORIZON}) ==")
env.run(until=WARMUP)
env.run(until=HORIZON)
print("== Fin simulación ==\n")

Ya que se corre la simulación, podemos hacer un reporte de que fue lo que pasó en la simulación. Esto es útil para poder analizar la relación entre los parámetros "alimentados" a la simulación y los resultados que arroja.

In [ ]:
def mean(x): 
    return sum(x)/len(x) if x else float('nan')

print(f"Pedidos servidos   : {served}")
print(f"Pedidos abandonados: {abandoned}")
print(f"Espera promedio    : {mean(wait_times):0.3f}")
print(f"Tiempo total (sojourn) promedio: {mean(sojourn_times):0.3f}")
print(f"Stock final        : {inventory.level:.0f}")